## Southern Ocean decadal variability

Assess the decadal variability of water mass transformation in the Southern Ocean, and relate it to changes in the overturning circulation.

In [1]:
import xarray as xr
import numpy as np
import gsw
from matplotlib import pyplot as plt
%matplotlib inline

### Load the data

In [2]:
# Grid data
vol4d = xr.open_dataset('~/so_decadal_variability/grid/SO_grid_vol4d_woa2018.nc',decode_times=False)#.isel(time=0)
grid = vol4d
dz = grid.depth_bnds.diff(dim='bnds')
# WOA hydrography decadal means
ct_WOA = xr.open_dataset('~/so_decadal_variability/ocean/SO_ocean_ct_woa2018_1981-2010.nc',decode_times=False)#.isel(time=[0,12])
sa_WOA = xr.open_dataset('~/so_decadal_variability/ocean/SO_ocean_sa_woa2018_1981-2010.nc',decode_times=False)#.isel(time=0)
gamman_WOA = xr.open_dataset('~/so_decadal_variability/ocean/SO_ocean_gamman_woa2018_1981-2010.nc',decode_times=False)#.isel(time=0)
alpha_WOA = xr.open_dataset('~/so_decadal_variability/ocean/SO_ocean_alpha_woa2018_1981-2010.nc',decode_times=False)#.isel(time=0)
beta_WOA = xr.open_dataset('~/so_decadal_variability/ocean/SO_ocean_beta_woa2018_1981-2010.nc',decode_times=False)#.isel(time=0)
# Merge
hydro_WOA = xr.merge([ct_WOA,sa_WOA,gamman_WOA,alpha_WOA,beta_WOA])
# Heat and freshwater flux
Q_ERA = -xr.open_dataset('~/so_decadal_variability/flux/SO_flux_ht_erai_1979-2018.nc').isel(time=range(360))
Qsw_ERA = -xr.open_dataset('~/so_decadal_variability/flux/SO_flux_sr_erai_1979-2018.nc').isel(time=range(360))
FW_ERA = xr.open_dataset('~/so_decadal_variability/flux/SO_flux_fw_erai_1979-2018.nc').isel(time=range(360))

# Set to have the same time
hydro_WOA = hydro_WOA.assign_coords(time=Q_ERA.time[0:12])
grid = grid.assign_coords(time=Q_ERA.time[0:12])

In [3]:
# Extend climatology
ct_WOA = xr.DataArray(np.tile(ct_WOA.ct,(30,1,1,1)),coords={'time':Q_ERA.time,'depth':hydro_WOA.depth,'lat':Q_ERA.lat,'lon':Q_ERA.lon},dims=['time','depth','lat','lon'])
ct_WOA.name = 'ct'
sa_WOA = xr.DataArray(np.tile(sa_WOA.sa,(30,1,1,1)),coords={'time':Q_ERA.time,'depth':hydro_WOA.depth,'lat':Q_ERA.lat,'lon':Q_ERA.lon},dims=['time','depth','lat','lon'])
sa_WOA.name = 'sa'
gamman_WOA = xr.DataArray(np.tile(gamman_WOA.gamman,(30,1,1,1)),coords={'time':Q_ERA.time,'depth':hydro_WOA.depth,'lat':Q_ERA.lat,'lon':Q_ERA.lon},dims=['time','depth','lat','lon'])
gamman_WOA.name = 'gamman'
alpha_WOA = xr.DataArray(np.tile(alpha_WOA.alpha,(30,1,1,1)),coords={'time':Q_ERA.time,'depth':hydro_WOA.depth,'lat':Q_ERA.lat,'lon':Q_ERA.lon},dims=['time','depth','lat','lon'])
alpha_WOA.name = 'alpha'
beta_WOA = xr.DataArray(np.tile(beta_WOA.beta,(30,1,1,1)),coords={'time':Q_ERA.time,'depth':hydro_WOA.depth,'lat':Q_ERA.lat,'lon':Q_ERA.lon},dims=['time','depth','lat','lon'])
beta_WOA.name = 'beta'
hydro_WOA = xr.merge([ct_WOA,sa_WOA,gamman_WOA,alpha_WOA,beta_WOA])

In [4]:
vol4d = xr.DataArray(np.tile(grid.vol4d,(30,1,1,1)),coords={'time':Q_ERA.time,'depth':hydro_WOA.depth,'lat':Q_ERA.lat,'lon':Q_ERA.lon},dims=['time','depth','lat','lon'])
vol4d.name = 'vol4d'

In [5]:
# EN4 hydrography monthly means
hydro_EN4 = xr.open_dataset('~/nilas/en4/v4.2.1/EN.4.2.1.f.analysis.g10.197901-201812.nc').isel(time=range(12))

## Depth penetration of shortwave radiation

Use empirical exponential decay function of Paulson and Simpson (1977)
$$ F_{PS77}(z) = R\ e^{-\frac{z}{h_1}}+(1-R)e^{-\frac{z}{h_2}} $$
where $R = 0.58$, $h_1 = 0.35m$, and $h_2 = 23m$.

In [6]:
R = 0.58
h1 = 0.35
h2 = 23

# Calculate in 1D
Fps77 = R*np.exp(-hydro_WOA.depth/h1)+(1-R)*np.exp(-hydro_WOA.depth/h2)
# Replicate over grid
Fps77 = Fps77*xr.ones_like(hydro_WOA.ct)

# Now get a 3D shortwave field
Qsw3d = Qsw_ERA.sr*Fps77
# and take the divergence
# (note that for taking the derivative, 'depth' must be positive upward, so reverse the sign)
# (subsequently, reverse the sign so that it is consistent with the other variables)
Qsw3d = Qsw3d.assign_coords(depth=-Qsw3d.depth) 
Qsw = Qsw3d.differentiate('depth')
Qsw = Qsw.assign_coords(depth=-Qsw.depth) 

## b-factor

From Jacket and McDougall (2005):
$$ b = \frac{|\nabla \gamma^n|}{|\nabla \rho_l|} $$

From Groeskamp et al. (2018):
$$ \nabla \rho_l = \rho (-\alpha\nabla \Theta + \beta \nabla S_A)$$
where $\rho$ is _in situ_ density.

In [7]:
def calc_b(T,S,rho,alpha,beta,gamma):
    # Calculate gradients in T, S and gamma
    # T
    gradx_T = T.differentiate('lon')
    grady_T = T.differentiate('lat')
    gradz_T = T.differentiate('depth')
    # S
    gradx_S = S.differentiate('lon')
    grady_S = S.differentiate('lat')
    gradz_S = S.differentiate('depth')
    # Locally referenced potential density
    gradx_r = rho*(-alpha*gradx_T + beta*gradx_S)
    grady_r = rho*(-alpha*grady_T + beta*grady_S)
    gradz_r = rho*(-alpha*gradz_T + beta*gradz_S)
    # gamma
    gradx_g = gamma.differentiate('lon')
    grady_g = gamma.differentiate('lat')
    gradz_g = gamma.differentiate('depth')

    # Calculate the absolute magnitudes
    absgradx_r = xr.ufuncs.sqrt(xr.ufuncs.square(gradx_r)+xr.ufuncs.square(grady_r)+xr.ufuncs.square(gradz_r))
    absgradx_g = xr.ufuncs.sqrt(xr.ufuncs.square(gradx_g)+xr.ufuncs.square(grady_g)+xr.ufuncs.square(gradz_g))
    
    # Calculate ratio
    b = absgradx_g/absgradx_r
    
    return b

In [8]:
# Calculate in situ density
z = hydro_WOA.depth*xr.ones_like(hydro_WOA.ct)
z = z.transpose('time','depth','lat','lon')
lat = hydro_WOA.lat*xr.ones_like(hydro_WOA.ct)
lat = lat.transpose('time','depth','lat','lon')
p = gsw.p_from_z(-z,lat)*xr.ones_like(hydro_WOA.ct).rename('p')
rho = gsw.rho(hydro_WOA.sa,hydro_WOA.ct,p)*xr.ones_like(hydro_WOA.ct).rename('rho')

In [9]:
# Calculate b
b = calc_b(hydro_WOA.ct,hydro_WOA.sa,rho,hydro_WOA.alpha,hydro_WOA.beta,hydro_WOA.gamman)

## Water mass transformation calculation

From Groeskamp and Iudicone (2018), transformation across a $\gamma^n$ surface is given by:
$$ G(\gamma^n) = \frac{\partial}{\partial \gamma^n}\iiint_{\hat{\gamma^n}\leq\gamma^n} \hat{\gamma^n}\ b\ F\ dV $$

The surface density flux, $F$, is given by:
$$ F = \big[-(E+P+R)S_A\beta - \frac{\alpha}{C_p}(Q_{lw}+Q_{lh}+Q_{sh})\big]\delta(z-\eta) + \frac{\alpha}{C_p}\nabla \cdot Q_{sw} F(z) $$
where $F(z)$ describes the vertical penetration of incoming shortwave radiation.

In [10]:
def calc_F(FW,Q,Qsw,S,alpha,beta,Cp=4200):
    
    Fheat = (alpha/Cp)*Q + (alpha/Cp)*Qsw
    Ffw = -FW*S*beta
    F = Fheat+Ffw
    
    F.name = 'F'
    Fheat.name = 'Fheat'
    Ffw.name = 'Ffw'
    
    return F, Fheat, Ffw

In [11]:
# Create a 3D mask with 1/dz in the surface and zero elsewhere
mask = xr.zeros_like(hydro_WOA.ct)
mask.loc[dict(depth=0)]=1/dz[0]
# Calculate the specific heat capacity of water

# Calculate density flux
F,Fheat,Ffw = calc_F(FW_ERA.fw*mask,Q_ERA.ht*mask,Qsw,hydro_WOA.sa,hydro_WOA.alpha,hydro_WOA.beta)

In [12]:
def calc_G(F,gamman,b,V,gn_edges):
    Gint = xr.DataArray(np.zeros(shape=(gn_edges.size,F.time.size)),coords={'gn': gn_edges,'time': F.time},dims=['gn','time'])
    gFbV = gamman*b*F*V
    i=0
    for g in gn_edges:
        mask = gamman<g
        Gint.loc[dict(gn=g)] = gFbV.where(mask,0).sum(dim=['lat','lon','depth'])
        i+=1
    G = Gint.differentiate('gn')
    
    return G

In [13]:
# Alternative, discrete volume calculation derived in Appendix 7.5 of Groeskamp et al (2018)
def calc_G_alt(F,gamman,b,V,gn_edges):
    gn_centres = (gn_edges[1:] + gn_edges[:-1]) / 2
    dgn = np.diff(gn_edges)
    G = xr.DataArray(np.zeros(shape=(gn_centres.size,F.time.size)),coords={'gn': gn_centres,'time': F.time},dims=['gn','time'])
    gFbV = gamman*b*F*V
    for i in range(gn_edges.size-1):
        mask = (gamman>gn_edges[i]) & (gamman<gn_edges[i+1])
        G.loc[dict(gn=gn_centres[i])] = gFbV.where(mask,0).sum(dim=['lat','lon','depth'])/dgn[i]
    
    return G

In [ ]:
# Calculate G
gn_edges = np.arange(26,28.5,0.1)
Gheat = calc_G(Fheat,hydro_WOA.gamman,b,vol4d,gn_edges)*1E-9
Gfw = calc_G(Ffw,hydro_WOA.gamman,b,vol4d,gn_edges)*1E-9
G = calc_G(F,hydro_WOA.gamman,b,vol4d,gn_edges)*1E-9

In [ ]:
Gheat.mean('time').plot(label='Gheat')
Gfw.mean('time').plot(label='Gfw')
G.mean('time').plot(label='G')
plt.legend()

In [ ]:
# Calculate G
gn_edges = np.arange(26,28.5,0.1)
Gheat = calc_G_alt(Fheat,hydro_WOA.gamman,b,grid.vol4d,gn_edges)*1E-9
Gfw = calc_G_alt(Ffw,hydro_WOA.gamman,b,grid.vol4d,gn_edges)*1E-9
G = calc_G_alt(F,hydro_WOA.gamman,b,grid.vol4d,gn_edges)*1E-9

In [ ]:
Gheat.mean('time').plot(label='Gheat')
Gfw.mean('time').plot(label='Gfw')
G.mean('time').plot(label='G')
plt.legend()